In [1]:
%matplotlib inline
%load_ext autoreload
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import time
from datetime import datetime
import scipy.optimize as opt
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
plt.style.use("ggplot")
mpl.pyplot.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams["font.size"] = 13
#mpl.rcParams["font.family"] = 'Osaka'

# import sys
# sys.path.append('../code/')

# from simulation_fixed_path import *
# from hash import *
# from plot_path import *

In [15]:
path = '../data/BTCdata_2.csv'
df_BTC = pd.read_csv(path, header=0)
df_BTC = df_BTC.rename(columns={'unixtime_precise': 'unixtime'})
df_BTC = df_BTC.loc[: ,['height', 'unixtime', 'Open', 'High', 'Low', 'Close', 'probability of success /Eh', 'expected return (USD/Eh)']]
df_BTC['time'] = pd.to_datetime(df_BTC['unixtime'].astype(int), unit='s')
df_BTC = df_BTC.sort_values(by='height', ascending=True)
df_BTC.head()

,height,unixtime,Open,High,Low,Close,probability of success /Eh,expected return (USD/Eh),time
0,450001,1485382343,896.66,896.66,895.43,895.44,0.000593,6.935004,2017-01-25 22:12:23
1,450002,1485383957,895.44,895.86,893.23,894.97,0.000593,7.424957,2017-01-25 22:39:17
2,450003,1485384299,895.44,895.86,893.23,894.97,0.000593,7.184180,2017-01-25 22:44:59
3,450004,1485384419,895.44,895.86,893.23,894.97,0.000593,7.025926,2017-01-25 22:46:59
4,450005,1485384847,895.44,895.86,893.23,894.97,0.000593,7.086069,2017-01-25 22:54:07


In [16]:
df_BTC.tail()

,height,unixtime,Open,High,Low,Close,probability of success /Eh,expected return (USD/Eh),time
119994,569995,1554279134,4962.68,4970.61,4925.26,4957.82,0.000036,2.470895,2019-04-03 08:12:14
119995,569996,1554281864,4957.82,4975.00,4929.67,4975.00,0.000036,2.514109,2019-04-03 08:57:44
119996,569997,1554282012,4957.82,4975.00,4929.67,4975.00,0.000036,2.482955,2019-04-03 09:00:12
119997,569998,1554282152,4957.82,4975.00,4929.67,4975.00,0.000036,2.451106,2019-04-03 09:02:32
119998,569999,1554282778,4957.82,4975.00,4929.67,4975.00,0.000036,2.453798,2019-04-03 09:12:58


In [17]:
df_BTC[0:30]

,height,unixtime,Open,High,Low,Close,probability of success /Eh,expected return (USD/Eh),time
0,450001,1485382343,896.66,896.66,895.43,895.44,0.000593,6.935004,2017-01-25 22:12:23
1,450002,1485383957,895.44,895.86,893.23,894.97,0.000593,7.424957,2017-01-25 22:39:17
2,450003,1485384299,895.44,895.86,893.23,894.97,0.000593,7.184180,2017-01-25 22:44:59
3,450004,1485384419,895.44,895.86,893.23,894.97,0.000593,7.025926,2017-01-25 22:46:59
4,450005,1485384847,895.44,895.86,893.23,894.97,0.000593,7.086069,2017-01-25 22:54:07
5,450006,1485384878,895.44,895.86,893.23,894.97,0.000593,6.871475,2017-01-25 22:54:38
6,450007,1485385013,895.44,895.86,893.23,894.97,0.000593,6.930437,2017-01-25 22:56:53
7,450008,1485385773,895.44,895.86,893.23,894.97,0.000593,7.109646,2017-01-25 23:09:33
8,450009,1485385825,895.44,895.86,893.23,894.97,0.000593,6.867119,2017-01-25 23:10:25
9,450010,1485387127,894.97,898.00,894.97,898.00,0.000593,7.250669,2017-01-25 23:32:07


* Simulation starts from the block with height 551443.

In [10]:
df_BTC[df_BTC['height'].isin([551443])]

,height,unixtime,probability of success /Eh,expected return (USD/Eh),time
101442,551443,1543137894,0.000035,1.584459,2018-11-25 09:24:54
